In [4]:
!pip install apscheduler dash datetime pytz plotly

from apscheduler.schedulers.background import BackgroundScheduler
import requests
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output
from datetime import datetime
import pytz

# Step 1: Setup the BLS API Request to Retrieve Data
def get_bls_data(series_id, start_year, end_year, api_key):
    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    headers = {"Content-Type": "application/json"}
    data = {
        "seriesid": [series_id],
        "startyear": str(start_year),
        "endyear": str(end_year),
        "registrationkey": api_key,
    }
    response = requests.post(url, json=data, headers=headers)
    json_data = response.json()

    if 'Results' in json_data:
        series_data = json_data['Results']['series'][0]['data']
        df = pd.DataFrame(series_data)
        df['value'] = df['value'].astype(float)
        df['date'] = pd.to_datetime(df['year'] + "-" + df['period'].str[1:] + "-01")
        df = df[['date', 'value']].sort_values('date')
        return df
    else:
        print("Error:", json_data.get('message', 'Unable to retrieve data'))
        return pd.DataFrame()

# Replace 'your_api_key' with your actual BLS API key
api_key = "f76aaf3f2b0a45569fb4cc8acfaf4738"

# Initialize data as global variables so they can be refreshed by the scheduler
u2_series_id = "LNS14023621"
u6_series_id = "LNS13327709"

u2_data = pd.DataFrame()
u6_data = pd.DataFrame()

def fetch_and_update_data():
    """Fetches data from BLS and updates global data variables."""
    global u2_data, u6_data
    # Fetch updated data for U-2 and U-6
    u2_data = get_bls_data(u2_series_id, start_year=2020, end_year=datetime.now().year, api_key="f76aaf3f2b0a45569fb4cc8acfaf4738")
    u6_data = get_bls_data(u6_series_id, start_year=2020, end_year=datetime.now().year, api_key="f76aaf3f2b0a45569fb4cc8acfaf4738")
    print(f"Data updated at {datetime.now()}")

# Initialize the scheduler
scheduler = BackgroundScheduler()

# Set scheduler to trigger on the first of each month at 8:31 AM EST
def scheduler():
    if trig == True:
        scheduler.add_job(
            func=fetch_and_update_data
        )
    else:
        scheduler.add_job(
            func=fetch_and_update_data,
            trigger="cron",
            day=1,
            hour=8,
            minute=31,
            timezone=pytz.timezone("America/New_York")
        )
    scheduler.start()

# Step 2: Set up the Dash App
app = Dash(__name__)

# Fetch initial data
fetch_and_update_data()

app.layout = html.Div([
    html.H1("BLS Jobs Dashboard"),
    html.Label("Select Unemployment Statistic:"),
    dcc.Dropdown(
        id="unemployment-stat",
        options=[
            {"label": "U-2 Unemployment", "value": "U2"},
            {"label": "U-6 Unemployment", "value": "U6"},
        ],
        value="U2",
    ),
    html.Label("Select Comparison Type:"),
    dcc.Dropdown(
        id="comparison-type",
        options=[
            {"label": "Month Over Month", "value": "MoM"},
            {"label": "Quarter Over Quarter", "value": "QoQ"},
            {"label": "Year Over Year", "value": "YoY"},
        ],
        value="MoM",
    ),
    dcc.Graph(id="jobs-graph"),
])

# Step 3: Define Callbacks for Updating the Graph
@app.callback(
    Output("jobs-graph", "figure"),
    Input("unemployment-stat", "value"),
    Input("comparison-type", "value"),
)
def update_graph(unemployment_stat, comparison_type):
    # Select the correct dataset based on the user's choice
    data = u2_data if unemployment_stat == "U2" else u6_data

    # Calculate the comparison based on selected type
    if comparison_type == "MoM":
        data["change"] = data["value"].pct_change(periods=1) * 100
        title = "Month Over Month % Change"
    elif comparison_type == "QoQ":
        data["change"] = data["value"].pct_change(periods=3) * 100
        title = "Quarter Over Quarter % Change"
    else:  # Year Over Year
        data["change"] = data["value"].pct_change(periods=12) * 100
        title = "Year Over Year % Change"

    # Prepare the figure
    fig = go.Figure(
        data=go.Scatter(
            x=data["date"], y=data["change"],
            mode="lines+markers",
            name=f"{unemployment_stat} Unemployment"
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="% Change in Unemployment",
        template="plotly_white"
    )

    return fig
    trig = True

# Run the App
if __name__ == "__main__":
    app.run_server(debug=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 9.2 MB/s eta 0:00:00
Data updated at 2024-11-09 19:23:12.667979


<IPython.core.display.Javascript object>